In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [1]:
!pip install datasets boto3 einops transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.8 MB/s eta 0:00:00


In [2]:
%%writefile ~/.passwd-s3fs


Writing /root/.passwd-s3fs


In [3]:
!chmod 600 ~/.passwd-s3fs
!apt install s3fs
!mkdir /s3
!s3fs baj40ja0abjabucketihvp /s3
!ls /s3/ihvp

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  mailcap mime-support
The following NEW packages will be installed:
  mailcap mime-support s3fs
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 325 kB of archives.
After this operation, 901 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 mailcap all 3.70+nmu1ubuntu1 [23.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 mime-support all 3.66 [3,696 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 s3fs amd64 1.90-1 [297 kB]
Fetched 325 kB in 1s (318 kB/s)
Selecting previously unselected package mailcap.
(Reading database ... 121654 files and directories currently installed.)
Preparing to unpack .../mailcap_3.70+nmu1ubuntu1_all.deb ...
Unpacking mailcap (3.70+nmu1ubuntu1) ...
Selecting previously unselected package mime-support.
Preparing

In [4]:
!cp /s3/ihvp/TinyStories_ihvp_f.pt /content/

In [5]:
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import torch

# Initialize the tokenizer
# tokenizer.pad_token = tokenizer.eos_token

# https://huggingface.co/datasets/Skylion007/openwebtext
from datasets import load_dataset

class TinyStoriesDataset(Dataset): # IterableDataset
    def __init__(self, split, tokenizer, path='roneneldan/TinyStories', block_size=1024, seed=42, take=10000, skip=0):
        self.split = split
        self.block_size = block_size
        self.tokenizer = tokenizer
        self.dataset = load_dataset(path, split=split, streaming=False) #.shuffle(seed=seed) #.skip(skip).take(take).with_format('torch')

    def __iter__(self):
        for item in self.dataset:
            text = item['text']
            encoding = self.tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True, padding='max_length', truncation=True, max_length=self.block_size)
            input_ids = encoding['input_ids'].squeeze()

            # Shift the input_ids and attention_mask to the right and pad
            labels = input_ids.clone()
            labels[:-1] = input_ids[1:]
            labels[-1] = self.tokenizer.pad_token_id

            yield {'input_ids': input_ids}, {'input_ids': labels}

    def __getitem__(self, index):
        item = self.dataset[index]
        text = item['text']
        encoding = self.tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True, padding='max_length', truncation=True, max_length=self.block_size)
        input_ids = encoding['input_ids'].squeeze()

        # Shift the input_ids and attention_mask to the right and pad
        labels = input_ids.clone()
        labels[:-1] = input_ids[1:]
        labels[-1] = self.tokenizer.pad_token_id

        return {'input_ids': input_ids}, {'input_ids': labels}

    def __len__(self):
      return len(self.dataset)

    def get_vocab_size(self):
        return self.tokenizer.vocab_size

    def get_block_size(self):
        return self.block_size


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch import nn

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer.pad_token = tokenizer.eos_token
vocab_size = tokenizer.vocab_size

model = AutoModelForCausalLM.from_pretrained("roneneldan/TinyStories-1M")

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

model.to(device)

tokenizer_config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/48.6M [00:00<?, ?B/s]

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 64)
    (wpe): Embedding(2048, 64)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-7): 8 x GPTNeoBlock(
        (ln_1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=64, out_features=64, bias=False)
            (v_proj): Linear(in_features=64, out_features=64, bias=False)
            (q_proj): Linear(in_features=64, out_features=64, bias=False)
            (out_proj): Linear(in_features=64, out_features=64, bias=True)
          )
        )
        (ln_2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=64, out_features=256, bias=True)
          (c_proj): Linear(in_features=256, out_

In [7]:
# 1. get a_l-1, use forward hook to save input to a layer l during the forward pass
layer_inputs = {}

def get_a_l_minus_1(name):

    def forward_hook_fn(module, input, output):
        layer_inputs[name] = torch.cat([
                input[0],
                torch.ones((input[0].shape[0], input[0].shape[1], 1)).to(input[0].device),
        ], dim=-1).clone().detach()
    return forward_hook_fn

# 2. get grad_loss, gradients of loss wrt output of linear transformation W_l a_l-1
#    using a backward hook on the linear layer that saves the gradient wrt the linear layer's output

layer_grads = {}
layer_grad_inputs = {}

def get_grad_loss(name):
    def back_hook_fn(module, grad_input, grad_output):
        print(grad_input[0].shape)
        layer_grads[name] = grad_output[0].clone().detach()
        # layer_grad_inputs[name] = grad_input[0].clone().detach()
        layer_grad_inputs[name] = torch.cat([
                    grad_input[0],
                    torch.ones((grad_input[0].shape[0], grad_input[0].shape[1], 1)).to(grad_input[0].device),
            ], dim=-1).clone().detach()
    return back_hook_fn

linear_layers = []
# specifc to GPT Neo
for name, module in model.named_modules():
    # INPUT TO MLP (batch_size, seq_len, input_dim), a_l-1
    if name.split('.')[-1] == 'mlp':
      print(name, module)
      module.register_forward_hook(get_a_l_minus_1(f"{name}.c_fc"))
      # linear_layers.append(module)
    # FIRST LINEAR LAYER self.linear(a_l_minus_1) proj
    # https://github.com/nrimsky/InfluenceFunctions/blob/72334413bde13cde66366d2e524f87f38d2adea2/mini_transformer.py#L98C1-L99C1
    if isinstance(module, nn.Linear) and name.split('.')[-1] == 'c_fc':
      print(name, module)
      module.register_full_backward_hook(get_grad_loss(name))
      linear_layers.append((name, module))
      layer_grads[name] = {}
      layer_inputs[name] = {}
      layer_grad_inputs[name] = {}

transformer.h.0.mlp GPTNeoMLP(
  (c_fc): Linear(in_features=64, out_features=256, bias=True)
  (c_proj): Linear(in_features=256, out_features=64, bias=True)
  (act): NewGELUActivation()
  (dropout): Dropout(p=0.0, inplace=False)
)
transformer.h.0.mlp.c_fc Linear(in_features=64, out_features=256, bias=True)
transformer.h.1.mlp GPTNeoMLP(
  (c_fc): Linear(in_features=64, out_features=256, bias=True)
  (c_proj): Linear(in_features=256, out_features=64, bias=True)
  (act): NewGELUActivation()
  (dropout): Dropout(p=0.0, inplace=False)
)
transformer.h.1.mlp.c_fc Linear(in_features=64, out_features=256, bias=True)
transformer.h.2.mlp GPTNeoMLP(
  (c_fc): Linear(in_features=64, out_features=256, bias=True)
  (c_proj): Linear(in_features=256, out_features=64, bias=True)
  (act): NewGELUActivation()
  (dropout): Dropout(p=0.0, inplace=False)
)
transformer.h.2.mlp.c_fc Linear(in_features=64, out_features=256, bias=True)
transformer.h.3.mlp GPTNeoMLP(
  (c_fc): Linear(in_features=64, out_features

In [8]:
model

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 64)
    (wpe): Embedding(2048, 64)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-7): 8 x GPTNeoBlock(
        (ln_1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=64, out_features=64, bias=False)
            (v_proj): Linear(in_features=64, out_features=64, bias=False)
            (q_proj): Linear(in_features=64, out_features=64, bias=False)
            (out_proj): Linear(in_features=64, out_features=64, bias=True)
          )
        )
        (ln_2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=64, out_features=256, bias=True)
          (c_proj): Linear(in_features=256, out_

In [9]:
def compute_grads(model: nn.Module, train_dataset: DataLoader):

    grads = [[] for _ in range(len(linear_layers))]
    full_tokenwise_grads = [[] for _ in range(len(linear_layers))]
    for X, Y in train_dataset:
        model.zero_grad()

        x_ids = X['input_ids'].to(device)
        y_ids = Y['input_ids'].to(device)

        if len(x_ids.shape) == 3:
            x_ids = x_ids.squeeze(1)
            y_ids = y_ids.squeeze(1)

        if len(x_ids.shape) == 1:
            x_ids = x_ids.unsqueeze(0)
            y_ids = y_ids.unsqueeze(0)

        output = model(x_ids, labels=y_ids)
        logits, loss = output['logits'], output['loss']

        loss.backward()
        for i, name_module in enumerate(linear_layers):
            name, module = name_module
            w_grad = module.weight.grad
            if module.bias is not None:
                b_grad = module.bias.grad.unsqueeze(-1)
                full_grad = torch.cat([w_grad, b_grad], dim=-1)
            else:
                full_grad = torch.cat(
                    [w_grad, torch.zeros([w_grad.shape[0], 1])],
                    dim=-1
                )

            d_s_l = layer_grads[name]
            a_l_1 = layer_inputs[name]
            tokenwise_grads = torch.einsum('bix,bjy->bixy', d_s_l, a_l_1)
            full_tokenwise_grads[i].append(tokenwise_grads.squeeze(0))

            grads[i].append(full_grad)

    return grads, full_tokenwise_grads

In [10]:
prompt = "Once upon a time, there was a girl name Alice. Alice enjoyed many things. One of the things Alice enjoyed was playing outside."
# prompt_enc = tokenizer.encode(prompt, return_tensors='pt').to(device)
# out = model.generate(prompt_enc, max_length=300)
# print(tokenizer.decode(out[0]))
# z_m_ids = torch.nn.functional.pad(out[0], (0, 2048 - len(out[0])), value=50256)

# z_m_label = z_m_ids.clone()
# z_m_label[:-1] = z_m_ids[1:]
# z_m_label[-1] = tokenizer.pad_token_id

# z_m = [({'input_ids': z_m_ids}, {'input_ids': z_m_label})]

prompts = [
    prompt,
    # """Emily and Benjamin were exploring the woods behind their house when they stumbled upon an old, weathered book. It was dusty and had a mysterious aura about it, sparking their curiosity instantly. Benjamin turned to Emily, "What should we do with the book?" Emily pondered for a moment, then declared, "Let's take it to Grandma and Grandpa!" With that, they dashed off, their hearts pounding with anticipation, eager to unveil their discovery. They burst into their grandparents' home, exclaiming, "Grandma, Grandpa! Look what we found!" Their grandparents were taken aback and inquired, "Where did you find this old book?""",
    # "Once upon a time, there was a small child named Lily. She enjoyed playing with her dolls and observing the butterflies in her garden. Lily had a large, fluffy bunny that she adored immensely. One day, Lily's bunny seemed gloomy. Lily couldn't understand why her bunny was feeling down.",
    # "The sun was beaming down as Peter emerged from his small cottage. He was a naive boy who took great pleasure in frolicking and playing games. He had an old rug that he used as a cape, often pretending to be a knight. Peter always longed for a playmate, but there was no one in sight. With a sigh, he resolved to seek out a companion. As he ventured out of the village, he overheard two voices, one of which was distinctly familiar. It was the voice of his neighbor, elderly Mrs. Smith. Peter crept closer, concealing himself behind a tree and peered through the leaves. Mrs. Smith and a stranger were in conversation, and the stranger appeared to be upset about something. Suddenly, the stranger raised his voice and advanced towards Mrs. Smith.",
    # """Frank the frog was very troubled. His fountain had broken and he was sure no one could fix it. One day Frank's best friend Sam the snail came to visit him. Sam asked, "What's wrong Frank?" Frank said sadly, "My fountain is broken and I can't fix it." But Sam had a plan! He said, "If you come with me I might know someone who can help you fix your fountain." So Frank and Sam went out into the woods, looking for help. And soon they found Snail Bob!""",
]

def process_prompts(prompts):
  z_m = []
  for prompt in prompts:
    prompt_enc = tokenizer.encode(prompt, return_tensors='pt').to(device)
    out = model.generate(prompt_enc, max_length=1000)
    print(tokenizer.decode(out[0], skip_special_tokens=True))
    z_m_ids = torch.nn.functional.pad(out[0], (0, 2048 - len(out[0])), value=50256)
    z_m_label = z_m_ids.clone()
    z_m_label[:-1] = z_m_ids[1:]
    z_m_label[-1] = tokenizer.pad_token_id

    z_m.append(({'input_ids': z_m_ids}, {'input_ids': z_m_label}))
  return z_m

z_m = process_prompts(prompts)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, there was a girl name Alice. Alice enjoyed many things. One of the things Alice enjoyed was playing outside. She was so excited to see what was inside.

Alice asked her mom, "Mommy, what is that?" Her mom replied, "It's a secret. It's a secret place to find it."

Alice was so excited. She opened the door and saw a big, beautiful flower. It was a beautiful flower. She was so excited to see it.

Alice and her mom went to the flower. It was so pretty and beautiful. Alice was so happy to see it. She couldn't wait to see it.

Alice and her mom went to the flower and saw that it was a beautiful flower. It was so pretty and beautiful. Alice was so happy that she had found the flower. She hugged it and said, "Thank you, mommy! I love it!"



In [11]:
import torch
ihvp = torch.load(f'TinyStories_ihvp_f.pt')

In [12]:
ihvp.shape

torch.Size([10000, 133120])

In [14]:
from torch.utils.data import DataLoader, Dataset
import random
from random import sample
random.seed(42)

def dataset_sample(dataset, n_samples):
    indices = sample(range(len(dataset)), n_samples)
    return [dataset[i] for i in indices]

dataset_length = 10000

train_dataset_hf = TinyStoriesDataset('train', tokenizer, block_size=2048, path='roneneldan/TinyStories')
train_dataset = dataset_sample(train_dataset_hf, dataset_length)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [15]:
topk = 5

all_top_training_samples = []
all_top_influences = []
all_top_token_influences = []

test_dataloader = DataLoader(z_m, batch_size=1)

for query, compl in z_m:
    # torch.Tensor(list(filter(lambda x: x != 50256, query['input_ids'].tolist()))).long()
    print(ihvp.shape)
    print(query)
    grads, token_grads = compute_grads(model, [(query, compl)])
    print("grads[0]", grads[0][0].view(-1).shape)
    print(token_grads[0][0].view(token_grads[0][0].shape[0], -1).shape)
    print('len(grads)', len(grads))
    print(len(token_grads))
    for l in grads:
        print("token_query_grads", l[0].view(-1).shape)
    query_grad = torch.cat(
        [q[0].view(-1) for q in grads]
    )
    token_query_grads = torch.cat(
        [tq[0].view(tq[0].shape[0], -1) for tq in token_grads],
        dim=-1
    )
    print("query_grad", query_grad.shape)
    print("token_query_grads", token_query_grads.shape)
    top_influences = -1 * torch.einsum("ij,j->i", ihvp, query_grad)
    print("top_influences", top_influences.shape)

    top_influences, top_samples = torch.topk(top_influences, topk)
    print(top_samples)
    all_top_training_samples.append(top_samples)
    all_top_influences.append(top_influences)
    all_top_token_influences.append(1)

    print("calculating tokenwise...")
    top_influence_tokenwise = []
    for i, r_t in enumerate(token_query_grads):
        top_influence_tokenwise.append(-1 * torch.einsum("ij,j->i", ihvp, r_t))

    print('len(top_influence_tokenwise)', len(top_influence_tokenwise))
    print('first top_influence_tokenwise', top_influence_tokenwise[0].shape)
    # for each of the 10000 examples, first top_influence_tokenwise is the
    # influence of the first token of each sample
    # when we get top_sample, for each token index in top_influence_tokenwise (len = 2048)
    # we get influence value for token at that sample index

top_influence_sentences = []
for k, (top_samples, top_influences) in enumerate(
        zip(all_top_training_samples, all_top_influences)
    ):
    influence_sentences = []
    for s, i in zip(top_samples, top_influences):
        s = s.item()
        query = tokenizer.decode(z_m[k][0]['input_ids']).replace("<|endoftext|>", "")
        print('query', query)
        sample = tokenizer.decode(train_dataset[s][0]['input_ids'], skip_special_tokens=True).replace("<|endoftext|>", "")
        print('sample', sample)

        tokenwise_influences = []
        for v, r_t in enumerate(top_influence_tokenwise):
            token_id = train_dataset[s][0]['input_ids'][v]
            token = tokenizer.decode(token_id, skip_special_tokens=True)
            influence = r_t[s].item()
            print(token, ":", influence)

            tokenwise_influences.append({"token": token, "influence": influence})
            # stop if encounter eot
            if token_id.item() == 50256:
                break

        influence_sentences.append({
            "sample": sample,
            "influence": i.item(),
            "tokens": tokenwise_influences
        })

    top_influence_sentences.append({"samples": influence_sentences, "query": query})


torch.Size([10000, 133120])
{'input_ids': tensor([ 7454,  2402,   257,  ..., 50256, 50256, 50256], device='cuda:0')}
torch.Size([1, 2048, 64])
torch.Size([1, 2048, 64])
torch.Size([1, 2048, 64])
torch.Size([1, 2048, 64])
torch.Size([1, 2048, 64])
torch.Size([1, 2048, 64])
torch.Size([1, 2048, 64])
torch.Size([1, 2048, 64])
grads[0] torch.Size([16640])
torch.Size([2048, 16640])
len(grads) 8
8
token_query_grads torch.Size([16640])
token_query_grads torch.Size([16640])
token_query_grads torch.Size([16640])
token_query_grads torch.Size([16640])
token_query_grads torch.Size([16640])
token_query_grads torch.Size([16640])
token_query_grads torch.Size([16640])
token_query_grads torch.Size([16640])
query_grad torch.Size([133120])
token_query_grads torch.Size([2048, 133120])
top_influences torch.Size([10000])
tensor([7134,  218,  924, 9902, 3668], device='cuda:0')
calculating tokenwise...
len(top_influence_tokenwise) 2048
first top_influence_tokenwise torch.Size([10000])
query Once upon a time, 

In [16]:
import json

with open('/content/influences.json', 'w') as outfile:
    json.dump({"influences": top_influence_sentences}, outfile,indent=4)

json.dumps({"influences": top_influence_sentences})

Query: Once upon a time, there was a girl name Alice. Alice enjoyed many things. One of the things Alice enjoyed was playing outside. She was so excited to see what was inside.

Alice asked her mom, "Mommy, what is that?" Her mom replied, "It's a secret. It's a secret place to find it."

Alice was so excited. She opened the door and saw a big, beautiful flower. It was a beautiful flower. She was so excited to see it.

Alice and her mom went to the flower. It was so pretty and beautiful. Alice was so happy to see it. She couldn't wait to see it.

Alice and her mom went to the flower and saw that it was a beautiful flower. It was so pretty and beautiful. Alice was so happy that she had found the flower. She hugged it and said, "Thank you, mommy! I love it!"
<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>